In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
import numpy as np

In [2]:
env = UE(file_name="Env/Test_Env/DroneFlightv1", seed=1, side_channels=[], no_graphics_monitor=False, no_graphics=False)
env = UPZBE(env)

In [3]:
def relocate_agents(env):
    return list(env.agents)  # simplified

# New helper to extract observation data for an agent
def get_agent_obs(obs, agent):
    agent_data = obs[agent]
    return np.array(agent_data[1]), np.array(agent_data[2])

agents = relocate_agents(env)

## Test PPO

In [ ]:
from PPO_Distillation.DistilledPPOAgent import DistilledPPO
from PPO_Distillation.Trajectories import ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params

c:\Users\rullo\anaconda3\envs\ml_agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# brain = DistilledPPO(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape, params['ppo_distilled'])
# brain.load_model()

## Test

In [6]:
steps = 0
brain.net.eval()
obs, done, t = env.reset(), [False for _ in env.agents], 0
episode_reward = 0
while not all(done) or t < 10000:
    actions = {}
    agents = relocate_agents()
    for agent in agents:
        if agent not in obs.keys():
            continue
        if isinstance(obs[agent], list):
            actions[agent], _, _ = brain.get_action(obs[agent][1], obs[agent][2])
        else:
            actions[agent], _, _ = brain.get_action(obs[agent]['observation'][1], obs[agent]['observation'][2])
        t += 1

    obs, reward, done, _ = env.step(actions)
    done = [done[agent] for agent in agents if agent in done.keys()]
    tot_reward = [reward[agent] for agent in agents if agent in reward.keys()]
mean_reward = np.mean(tot_reward)
print(f"Mean reward: {mean_reward}")

Mean reward: -0.5004000663757324


# Test SAC

In [4]:
from SAC_Distillation.DistilledSACAgent import DistilledSAC
from SAC_Distillation.Trajectories import SAC_ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params

In [5]:
brain = DistilledSAC(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape,len(agents), params['sac_distilled'])
brain.load('SavedModels/SAC_distilled_trained.pth')

c:\Users\rullo\Desktop\Master_Thesis\Project\Dreamer\SAC_Distillation\DistilledSACAgent.py:259: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torc

In [6]:
num_test_episodes = 10
max_episode_steps = 2048

for episode in range(num_test_episodes):
    obs = env.reset()
    done = {agent: False for agent in env.agents}
    episode_reward, t = {agent: 0 for agent in env.agents}, 0

    while not all(done.values()) and t < max_episode_steps:
        actions = {}

        for agent in env.agents:
            if agent not in obs:
                continue

            camera_obs, vector_obs = get_agent_obs(obs, agent)

            action = brain.get_action(camera_obs, vector_obs)
            actions[agent] = action

        next_obs, rewards, dones, _ = env.step(actions)

        for agent in rewards:
            episode_reward[agent] += rewards[agent]

        obs = next_obs
        done = dones
        t += 1

    avg_episode_reward = np.mean(list(episode_reward.values()))
    print(f"Episode {episode + 1}/{num_test_episodes} | Average Reward: {avg_episode_reward:.2f} | Steps: {t}")

env.close()

Episode 1/10 | Average Reward: -1.00 | Steps: 2000


KeyboardInterrupt: 